In [2]:
## What-If Tool from scratch - From CSV to trained binary classification model to What-If Tool usage

# This notebook shows the process of loading up a dataset from CSV, training a very simple classifier to
# predict one of the columns, then using the What-If Tool (WIT) to analyze the training dataset and the trained
# model.

# This notebook uses the UCI Census dataset and learning problem, detailed at
# https://archive.ics.uci.edu/ml/datasets/census+income, which predicts whether a person earns more than $50k
# given their census information.
# To customize this notebook to work on your own dataset, you only need to edit the sections marked with "USER: "

## Setup (install Jupyter, Tensorflow, and Tensorflow Serving in a virtualenv).
# NOTE: Use of a virtualenv, pip installation of tensorflow and docker use for TF Serving aren't the only way
# to set all this up. I just find it the simplest and safest to use.

# Step 1: Install Tensorflow using pip/virtualenv - See https://www.tensorflow.org/install/pip for instructions

# Step 2: Install Tensorflow Serving using docker - See https://www.tensorflow.org/serving/docker for instructions

# The next steps must be done from a terminal that has activated the virtualenv that was created in step 1

# Step 3: Install Jupyter and pandas to view and run this notebook
# > pip install jupyter
# > pip install pandas

# Step 4: Run this notebook
# > jupyter notebook
# From the file selector that opens in the browser, select this notebook file.
# Run the cells.

In [3]:
## Load helper functions

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import data

# Writes a pandas dataframe to disk as a tfrecord file of tf.Example protos,
# using only the dataframe columns specified. Non-numeric columns are treated
# as strings.
def write_df_as_tfrecord(df, filename, columns=None):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    writer = tf.python_io.TFRecordWriter(filename)
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(row[col])
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        writer.write(example.SerializeToString())
    writer.close()


# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns):
    feature_spec = {}
    for f in columns:
        if df[f].dtype is np.dtype(np.int64):
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

# Parses a serialized tf.Example into input features and target feature from 
# the provided label feature name and feature spec.
def parse_tf_example(example_proto, label, feature_spec):
    parsed_features = tf.parse_example(serialized=example_proto, features=feature_spec)
    target = parsed_features.pop(label)
    return parsed_features, target

# An input function for providing input to a model from tf.Examples from tf record files.
def tfrecords_input_fn(files_name_pattern, feature_spec, label, mode=tf.estimator.ModeKeys.EVAL,
                       num_epochs=None, 
                       batch_size=64):
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    file_names = tf.matching_files(files_name_pattern)
    dataset = data.TFRecordDataset(filenames=file_names)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example, label, feature_spec))
    dataset = dataset.repeat(num_epochs)
    return dataset

# Creates simple numeric and categorical feature columns from a feature spec and a
# list of columns from that spec to use.
#
# NOTE: Models might perform better with some feature engineering such as bucketed
# numeric columns and hash-bucket/embedding columns for categorical features.
def create_feature_columns(columns, feature_spec):
    ret = []
    for col in columns:
        if feature_spec[col].dtype is tf.int64 or feature_spec[col].dtype is tf.float32:
            ret.append(tf.feature_column.numeric_column(col))
        else:
            ret.append(tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_vocabulary_list(col, list(df[col].unique()))))
    return ret

In [4]:
## Read the dataset from a CSV into dataframe and display a list of all columns and a preview of the data

# USER: Set the path to the CSV containing the dataset to train on (can be a web address or local path).
csv_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# USER: Set the column names for the columns in the CSV. If the CSV's first line is a header line containing
# the column names, then set this to None.
csv_columns = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital-Status",
               "Occupation", "Relationship", "Race", "Sex", "Capital-Gain", "Capital-Loss",
               "Hours-per-week", "Country", "Target"]

# Read the dataset from the provided CSV and print out information about it.
df = pd.read_csv(csv_path, names=csv_columns, skipinitialspace=True)
print df.columns.tolist()
df

['Age', 'Workclass', 'fnlwgt', 'Education', 'Education-Num', 'Marital-Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital-Gain', 'Capital-Loss', 'Hours-per-week', 'Country', 'Target']


,Age,Workclass,fnlwgt,Education,Education-Num,Marital-Status,Occupation,Relationship,Race,Sex,Capital-Gain,Capital-Loss,Hours-per-week,Country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [5]:
# USER: Set the name you want to give the directory the model will be saved to
model_name = 'trained_model'

# USER: Set the name you want to give the tfrecord dataset file
tfrecord_name = 'data.tfrecord'

# USER: Set the column in the dataset you wish for the model to predict
label_column = 'Target'

model_path = os.path.join(os.getcwd(), model_name)
tfrecord_path = os.path.join(os.getcwd(), tfrecord_name)

# USER: Make the label column numeric (0 and 1), for use in our model.
# In this case, examples with a target value of '<=50K' are considered to be in the '0' (negative) class
# and all other examples are considered to be in the '1' (positive) class.
df[label_column] = np.where(df[label_column] == '<=50K', 0, 1)

# USER: If the CSV needs any clean-up (such as removing problematic rows or creating new columns), do it here.

# USER: Set list of all columns from the dataset we will use for model input.
input_features = ['Age', 'Workclass', 'Education', 'Marital-Status', 'Occupation', 'Relationship', 'Race', 'Sex',
                  'Capital-Gain', 'Capital-Loss', 'Hours-per-week', 'Country']

# Ensure the label column is not accidentally set as an input feature.
if label_column in input_features:
    input_features.remove(label_column)

# Create a list containing all input features and the label column
features_and_labels = input_features + [label_column]

In [6]:
# Write the records to disk as tf.Example protos in tf record file, for use in model training
# and later for use by WIT.
write_df_as_tfrecord(df, tfrecord_path, features_and_labels)

In [7]:
## Create and train the classifier

import functools

# Create a feature spec for the classifier
feature_spec = create_feature_spec(df, features_and_labels)

# Define and train the classifier

train_inpf = functools.partial(tfrecords_input_fn, tfrecord_path, feature_spec, label_column)
classifier = tf.estimator.LinearClassifier(
    feature_columns=create_feature_columns(input_features, feature_spec))
classifier.train(train_inpf, steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe8845dc390>, '_model_dir': '/tmp/tmpxSSPfU', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
INFO:tenso

INFO:tensorflow:global_step/sec: 312.846
INFO:tensorflow:loss = 83.02547, step = 6801 (0.316 sec)
INFO:tensorflow:global_step/sec: 438.864
INFO:tensorflow:loss = 27.039501, step = 6901 (0.228 sec)
INFO:tensorflow:global_step/sec: 415.462
INFO:tensorflow:loss = 20.402008, step = 7001 (0.241 sec)
INFO:tensorflow:global_step/sec: 499.366
INFO:tensorflow:loss = 26.103325, step = 7101 (0.200 sec)
INFO:tensorflow:global_step/sec: 484.672
INFO:tensorflow:loss = 54.734047, step = 7201 (0.206 sec)
INFO:tensorflow:global_step/sec: 469.305
INFO:tensorflow:loss = 43.70603, step = 7301 (0.213 sec)
INFO:tensorflow:global_step/sec: 484.552
INFO:tensorflow:loss = 63.85334, step = 7401 (0.207 sec)
INFO:tensorflow:global_step/sec: 440.298
INFO:tensorflow:loss = 93.848854, step = 7501 (0.227 sec)
INFO:tensorflow:global_step/sec: 461.159
INFO:tensorflow:loss = 54.2349, step = 7601 (0.217 sec)
INFO:tensorflow:global_step/sec: 354.274
INFO:tensorflow:loss = 50.982765, step = 7701 (0.282 sec)
INFO:tensorflow

In [8]:
classifier.predict(input_fn=train_inpf)

<generator object predict at 0x7fe849754870>

In [9]:
#predictions = np.array([item['probabilities'][0] for item in x])

In [10]:
## Save the classifier to disk for serving

# Uses a parsing serving input receiver function so that it can classify from serialized tf.Examples
# using the TensorFlow Serving Classify API.

serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
classifier.export_savedmodel(model_path, serving_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Restoring parameters from /tmp/tmpxSSPfU/model.ckpt-10000
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /d/Desktop/UCL/Coursework/SecondYear/SystemsEngineering/ProgressChecking/trained_model/temp-1549132818/saved_model.pb


'/d/Desktop/UCL/Coursework/SecondYear/SystemsEngineering/ProgressChecking/trained_model/1549132818'

In [11]:
## Print out the What-If Tool usage instructions (serve model, launch TensorBoard, configure What-If Tool)
import urllib

docker_command = 'docker run -p 8500:8500 --mount type=bind,source=%s,target=/models/my_model/ -e MODEL_NAME=my_model -t tensorflow/serving' % model_path
what_if_tool_path = ('http://localhost:6006/#whatif&inferenceAddress1=%s&modelName1=my_model&examplesPath=%s' % 
                     (urllib.quote('localhost:8500'), urllib.quote(tfrecord_path)))

print 'Command to serve model (depending on your docker installation, you may need to add "sudo" to the front of the docker command):'
print docker_command
print '\n'


print 'Command to launch tensorboard:'
print 'tensorboard --logdir .'
print '\n'


print 'URL to view What-If Tool for your model and dataset:'
print what_if_tool_path

# To kill the served model, find the docker container ID through 'docker container ls',
# then run 'docker kill [containerId]' (run as sudo as necessary)

Command to serve model (depending on your docker installation, you may need to add "sudo" to the front of the docker command):
docker run -p 8500:8500 --mount type=bind,source=/d/Desktop/UCL/Coursework/SecondYear/SystemsEngineering/ProgressChecking/trained_model,target=/models/my_model/ -e MODEL_NAME=my_model -t tensorflow/serving


Command to launch tensorboard:
tensorboard --logdir .


URL to view What-If Tool for your model and dataset:
http://localhost:6006/#whatif&inferenceAddress1=localhost%3A8500&modelName1=my_model&examplesPath=/d/Desktop/UCL/Coursework/SecondYear/SystemsEngineering/ProgressChecking/data.tfrecord
